In [1]:
import coverage

## Set API Key

In [9]:
%env OPENAI_API_KEY=sk-proj-2bukD0FHxc5cMV1KaW90T3BlbkFJ6nfZBZlPJc9SnImkUysE

env: OPENAI_API_KEY=sk-proj-2bukD0FHxc5cMV1KaW90T3BlbkFJ6nfZBZlPJc9SnImkUysE


## Coverage Report

In [2]:
!coverage run -m pytest test_main.py
!coverage report -m

============================= test session starts ==============================
platform darwin -- Python 3.9.12, pytest-7.1.1, pluggy-1.0.0
rootdir: /Users/rui/Documents/GitHub/CSE291Gcodecoverage
plugins: anyio-3.5.0
collected 3 items                                                              

test_main.py ...                                                         [100%]

============================== 3 passed in 0.01s ===============================
Name           Stmts   Miss  Cover   Missing
--------------------------------------------
main.py           21      7    67%   16-19, 23, 26-27
test_main.py      16      0   100%
--------------------------------------------
TOTAL             37      7    81%


In [3]:
cov = coverage.Coverage()
cov.load()
cov.analysis('main.py')[2]

[16, 17, 18, 19, 23, 26, 27]

## Get Code

In [4]:
code = {}
with open('main.py') as f:
    for i, line in enumerate(f, 1):
        # get line until newline character
        line = line.rstrip()
        code[i] = line
print(code)

{1: 'from dataclasses import dataclass', 2: '', 3: '@dataclass()', 4: 'class Product:', 5: '', 6: '    id: int', 7: '    name: str', 8: '    price: float', 9: '    stock: int', 10: '', 11: '    def increase_stock(self, stock_to_add: int):', 12: '        self.check_positive_number(stock_to_add)', 13: '        self.stock: int = self.stock + stock_to_add', 14: '', 15: '    def decrease_stock(self, stock_to_reduce):', 16: '        self.check_positive_number(stock_to_reduce)', 17: '        new_stock = self.stock - stock_to_reduce', 18: '        self.check_negative_stock(new_stock)', 19: '        self.stock = self.stock - stock_to_reduce', 20: '', 21: '    def check_positive_number(self, value):', 22: '        if value <= 0:', 23: '            raise Exception("Number must be positive")', 24: '', 25: '    def check_negative_stock(self, value):', 26: '        if value < 0:', 27: '            raise Exception("Stock must be greater than or equal to 0")', 28: ''}


In [5]:
test_code = {}
with open('test_main.py') as f:
    for i, line in enumerate(f, 1):
        # get line until newline character
        line = line.rstrip()
        test_code[i] = line
print(test_code)

{1: 'from main import Product', 2: 'from dataclasses import is_dataclass', 3: 'import pytest', 4: '', 5: 'def test_if_it_is_a_dataclass():', 6: '    assert is_dataclass(Product) == True', 7: '', 8: '', 9: '@pytest.fixture', 10: 'def product():', 11: "    return Product(1, 'foo', 1.0, 10)", 12: '', 13: '', 14: 'def test_constructor(product):', 15: '    assert product.id == 1', 16: "    assert product.name == 'foo'", 17: '    assert product.price == 1.0', 18: '    assert product.stock == 10', 19: '', 20: '', 21: 'def test_increase_stock(product):', 22: '    product.increase_stock(10)', 23: '    assert product.stock == 20'}


In [6]:
code_txt = "\n".join(code.values())
test_code_txt = "\n".join(test_code.values())

In [7]:
# aggregate missing lines according to the coverage report
missing_lines = []
for line in cov.analysis('main.py')[2]:
    missing_lines.append(code[line])
missing_lines_txt = "\n".join(missing_lines)

## Baseline: given code and a test example, generate test case (GPT 4-o)

In [10]:
from openai import OpenAI
client = OpenAI()

In [11]:
gpt_prompt = "Given the following code, generate test cases. Only respond with code that is valid Python code starting with code: from main import Product from dataclasses import is_dataclass import pytest @pytest.fixture def product(): return Product(1, 'foo', 1.0, 10) def test_constructor(product): assert product.id == 1 assert product.name == 'foo'"
print(gpt_prompt)

Given the following code, generate test cases. Only respond with code that is valid Python code starting with code: from main import Product from dataclasses import is_dataclass import pytest @pytest.fixture def product(): return Product(1, 'foo', 1.0, 10) def test_constructor(product): assert product.id == 1 assert product.name == 'foo'


In [12]:
message=[{"role": "system", "content": "You are a helpful programmer writing test.py."},{"role": "assistant", "content": gpt_prompt + "\n" + "\n".join(code.values())}]


response = client.chat.completions.create(
    model="gpt-4o",
    messages = message,
)
# print(response.choices[0].message)

In [13]:
responseText = response.choices[0].message.content
# get rid of the first 10 characters and the last 3 characters
responseText = responseText[10:]
responseText = responseText[:-3]
responseText

'from main import Product\nfrom dataclasses import is_dataclass\nimport pytest\n\n@pytest.fixture\ndef product():\n    return Product(1, \'foo\', 1.0, 10)\n\ndef test_constructor(product):\n    assert product.id == 1\n    assert product.name == \'foo\'\n    assert product.price == 1.0\n    assert product.stock == 10\n\ndef test_increase_stock(product):\n    product.increase_stock(5)\n    assert product.stock == 15\n\ndef test_decrease_stock(product):\n    product.decrease_stock(5)\n    assert product.stock == 5\n\ndef test_decrease_stock_exception(product):\n    with pytest.raises(Exception, match="Stock must be greater than or equal to 0"):\n        product.decrease_stock(20)\n\ndef test_increase_stock_with_zero(product):\n    with pytest.raises(Exception, match="Number must be positive"):\n        product.increase_stock(0)\n\ndef test_increase_stock_with_negative_number(product):\n    with pytest.raises(Exception, match="Number must be positive"):\n        product.increase_stock(-5)\

In [14]:
# if base_test.py already exists, delete it
import os
if os.path.exists('base_test.py'):
    os.remove('base_test.py')
# store the response in a file with name response.py
with open('base_test.py', 'w') as f:
    f.write(responseText)

In [15]:
!coverage run -m pytest base_test.py
!coverage report -m

============================= test session starts ==============================
platform darwin -- Python 3.9.12, pytest-7.1.1, pluggy-1.0.0
rootdir: /Users/rui/Documents/GitHub/CSE291Gcodecoverage
plugins: anyio-3.5.0
collected 8 items                                                              

base_test.py ........                                                    [100%]

============================== 8 passed in 0.02s ===============================
Name           Stmts   Miss  Cover   Missing
--------------------------------------------
base_test.py      32      0   100%
main.py           21      0   100%
--------------------------------------------
TOTAL             53      0   100%


## Baseline + Coverage

In [16]:
from openai import OpenAI
client = OpenAI()

In [23]:
gpt_prompt = "Given the following code, test cases, and coverage report on the lines missing test coverage, generate more test cases to obtain more test coverage. Only respond with valid Python code that can be appended to the test cases.\n Code:\n" + code_txt +"\n Test Cases:\n" + test_code_txt + "\n Missing Lines from Coverage Report:\n" + missing_lines_txt + "\n"
print(gpt_prompt)

Given the following code, test cases, and coverage report on the lines missing test coverage, generate more test cases to obtain more test coverage. Only respond with valid Python code that can be appended to the test cases.
 Code:
from dataclasses import dataclass

@dataclass()
class Product:

    id: int
    name: str
    price: float
    stock: int

    def increase_stock(self, stock_to_add: int):
        self.check_positive_number(stock_to_add)
        self.stock: int = self.stock + stock_to_add

    def decrease_stock(self, stock_to_reduce):
        self.check_positive_number(stock_to_reduce)
        new_stock = self.stock - stock_to_reduce
        self.check_negative_stock(new_stock)
        self.stock = self.stock - stock_to_reduce

    def check_positive_number(self, value):
        if value <= 0:
            raise Exception("Number must be positive")

    def check_negative_stock(self, value):
        if value < 0:
            raise Exception("Stock must be greater than or equ

In [24]:
message=[{"role": "system", "content": "You are a helpful programmer coding test.py. Generate only python test cases that can be appended to: \n"+test_code_txt},{"role": "assistant", "content": gpt_prompt + "\n" + "\n".join(code.values())}]


response = client.chat.completions.create(
    model="gpt-4o",
    messages = message,
)
# print(response.choices[0].message)


In [25]:
responseText = response.choices[0].message.content
# get rid of the first 10 characters and the last 3 characters
responseText = responseText[10:]
responseText = responseText[:-3]
responseText

'def test_decrease_stock(product):\n    product.decrease_stock(5)\n    assert product.stock == 5\n\n    product.decrease_stock(1)\n    assert product.stock == 4\n\ndef test_decrease_stock_too_much(product):\n    with pytest.raises(Exception, match="Stock must be greater than or equal to 0"):\n        product.decrease_stock(11)\n\ndef test_decrease_stock_negative_value(product):\n    with pytest.raises(Exception, match="Number must be positive"):\n        product.decrease_stock(-5)\n\ndef test_check_positive_number_negative_value(product):\n    with pytest.raises(Exception, match="Number must be positive"):\n        product.check_positive_number(-1)\n\ndef test_check_negative_stock(product):\n    with pytest.raises(Exception, match="Stock must be greater than or equal to 0"):\n        product.check_negative_stock(-1)\n'

In [26]:
# if cov_test.py already exists, delete it
import os
if os.path.exists('cov_test.py'):
    os.remove('cov_test.py')
# copy the test_main file to cov.py
!cp test_main.py cov_test.py
# append the response to the cov.py file
with open('cov_test.py', 'a') as f:
    f.write(responseText)

In [27]:
!coverage run -m pytest cov_test.py
!coverage report -m

============================= test session starts ==============================
platform darwin -- Python 3.9.12, pytest-7.1.1, pluggy-1.0.0
rootdir: /Users/rui/Documents/GitHub/CSE291Gcodecoverage
plugins: anyio-3.5.0
collected 8 items                                                              

cov_test.py ........                                                     [100%]

============================== 8 passed in 0.02s ===============================
Name          Stmts   Miss  Cover   Missing
-------------------------------------------
cov_test.py      33      0   100%
main.py          21      0   100%
-------------------------------------------
TOTAL            54      0   100%
